In [1]:
from datetime import *
import gspread
from dateutil.parser import parse
from oauth2client.service_account import ServiceAccountCredentials
import random
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [16]:
class ContasPagar:

    def __init__(self):
        scope = ['https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(
            'fluxo-de-caixa-dd17434e5a66.json', scope)
        client = gspread.authorize(creds)
        sheet = client.open('contas-a-pagar').sheet1
        self.sheet = sheet

    def head(self):
        indice = self.sheet.row_values(1)
        self.indice = indice
        return self.indice

    #funcionando
    def data_compra(self):
        data = self.sheet.col_values(1)
        mes_compra = []
        row = 0
        # converte o formato de string em data
        for item in data[2:]:
            if len(item) > 0:
                item_data = datetime.strptime(item, '%d/%m/%Y')
                mes_compra.insert(row, item_data)
                row += 1
            else:
                pass
        self.mes_compra = mes_compra
        return self.mes_compra

    def data_vencimento(self):
        data = self.sheet.col_values(1)
        mes_compra = []
        row = 0
        # converte o formato de string em data
        for item in data[2:]:
            if len(item) > 0:
                item_data = datetime.strptime(item, '%d/%m/%Y')
                mes_compra.insert(row, item_data)
                row += 1
            else:
                pass
        self.mes_compra = mes_compra
        return self.mes_compra

    def fornecedor(self):
        data = self.sheet.col_values(3)
        fornecedor = []
        row = 0
        for el in data[2:]:
            if len(el) > 0:
                fornecedor.insert(row, el)
                row += 1
            else:
                pass
        self.fornecedor = fornecedor
        return self.fornecedor

    def valor_boletos(self):
        # recebe o valor do boleto devido
        data = self.sheet.col_values(4)
        # conv = converte_data(data)
        self.data = data
        valor_boletos = []
        #passar dados para float
        row = 0
        total_boletos = 0
        # Inicia no valor inicial 2 linha
        for item in data[2:]:
            if len(item) > 0:
                item = item.replace('.', '')
                item = item.replace(',', '.')
                valor = float(item)
                valor_boletos.insert(row, valor)
                row += 1
            else:
                pass
        self.valor_boletos = valor_boletos
        return self.valor_boletos

    def banco(self):
        data = self.sheet.col_values(5)
        row = 0
        banco = []
        for el in data[2:]:
            if len(el) > 0:
                banco.insert(row, el)
                row += 1
            else:
                pass
        self.banco = banco
        return self.banco


    def status(self):
        data = self.sheet.col_values(6)
        status = []
        row = 0
        for el in status:
            status.insert(row, el)
            row += 1
        else:
            pass
        self.status = status
        return self.status

    def panda(self):
        dc = pd.Series(c.data_compra(), name='data_compra')
        dv = pd.Series(c.data_vencimento(), name='data_vencimento')
        f = pd.Series(c.fornecedor(), name='fornecedor')
        vb = pd.Series(c.valor_boletos(), name='valor_boletos')
        b = pd.Series(c.banco(), name='banco')
        s = pd.Series(c.status(), name='status')
        df = pd.concat([dc, dv, f, vb, b, s], axis=1)
        self.df = df
        return self.df


In [17]:
c = ContasPagar()

In [6]:
print(c.banco)


['BB', 'BB', 'BB', 'bb', 'itau', 'bradesco', 'bradesco', 'santander', 'BB', 'santander']


In [18]:
df = c.panda()


In [21]:
df.banco



0           BB
1           BB
2           BB
3           bb
4         itau
5     bradesco
6     bradesco
7    santander
8           BB
9    santander
Name: banco, dtype: object

In [23]:
df



,data_compra,data_vencimento,fornecedor,valor_boletos,banco,status
0,2017-07-25,2017-07-25,Lb - Ind,1142.82,BB,NaN
1,2017-07-25,2017-07-25,Lb - Ind,1142.82,BB,NaN
2,2017-07-25,2017-07-25,Lb - Ind,1142.86,BB,NaN
3,2017-07-25,2017-07-25,Lb - Ind,1142.86,bb,NaN
4,2017-08-10,2017-08-10,Fava,653.60,itau,NaN
5,2017-08-25,2017-08-25,Total Formento,1818.80,bradesco,NaN
6,2017-08-25,2017-08-25,Total Formento,1617.64,bradesco,NaN
7,2017-08-11,2017-08-11,Rhr Cosmésticos,326.13,santander,NaN
8,2017-07-25,2017-07-25,Lb - Ind,1142.86,BB,NaN
9,2017-08-11,2017-08-11,Rhr Cosmésticos,326.13,santander,NaN


In [25]:
#aplica query
df.query('fornecedor == "Total Formento"')

,data_compra,data_vencimento,fornecedor,valor_boletos,banco,status
5,2017-08-25,2017-08-25,Total Formento,1818.80,bradesco,NaN
6,2017-08-25,2017-08-25,Total Formento,1617.64,bradesco,NaN


In [29]:
df.query('fornecedor == "Total Formento"').groupby('data_vencimento').mean()


,valor_boletos,status
data_vencimento,,
2017-08-25,1718.22,NaN


In [31]:
df


,data_compra,data_vencimento,fornecedor,valor_boletos,banco,status
0,2017-07-25,2017-07-25,Lb - Ind,1142.82,BB,NaN
1,2017-07-25,2017-07-25,Lb - Ind,1142.82,BB,NaN
2,2017-07-25,2017-07-25,Lb - Ind,1142.86,BB,NaN
3,2017-07-25,2017-07-25,Lb - Ind,1142.86,bb,NaN
4,2017-08-10,2017-08-10,Fava,653.60,itau,NaN
5,2017-08-25,2017-08-25,Total Formento,1818.80,bradesco,NaN
6,2017-08-25,2017-08-25,Total Formento,1617.64,bradesco,NaN
7,2017-08-11,2017-08-11,Rhr Cosmésticos,326.13,santander,NaN
8,2017-07-25,2017-07-25,Lb - Ind,1142.86,BB,NaN
9,2017-08-11,2017-08-11,Rhr Cosmésticos,326.13,santander,NaN


In [34]:
dir(df)



['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_SLICEMAP',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_wrap__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__